# Dev Notebook

In [ ]:
# v0.1 DS FRAME

# singleton = string
# proposition = list of strings


from collections import defaultdict
import copy


class DSFrame:
    """
    A class to represent a DS-Theoretic Frame
    """

    def __init__(self, singletons):
        """
        Constructor
        """
        # List of lowercased singletons of size n. The index of these singletons is important.
        self._singletons = [x.lower() for x in singletons]
        self._size_singletons = len(self._singletons)
        self._size_powerset = self._size_singletons**2

        # dict containing masses (unnormalized)
        # this is also the masses DSVector
        self._dsvector = defaultdict(self._default_mass)

        # Lookup table containing keys of singletons in the masses_dsvector
        self._power = self._initialize_power()

    def _default_mass(self):
        return 0

    @property
    def singletons(self):
        """
        Get singletons
        """
        return self._singletons

    @property
    def power(self):
        """
        Get power
        """
        return self._power

    def get_dsvector(self):
        """
        Get core (masses_dsvector)
        """
        return self._dsvector

    def set_dsvector(self, index, mass):
        """
        Set core
        """
        self._dsvector.update({index: mass})

    def get_mass(self, proposition):
        """
        Get mass for a proposition or a singleton
        """
        proposition = [x.lower() for x in proposition]
        index = self._get_index_from_dsvector(proposition)
        return self.get_dsvector()[index]

    def get_normalized_mass(self, proposition):
        """
        Get normalized mass for a proposition
        """
        proposition = [x.lower() for x in proposition]
        return self.get_mass(proposition) / self._get_normalizing_constant()

    def get_normalized_dsvector(self):
        """
        Returns a dsvector with all the masses normalized
        """
        normalized_masses = {}
        for key, value in self.get_dsvector().items():
            normalized_mass = value / self._get_normalizing_constant()
            normalized_masses.update({key: normalized_mass})
        return normalized_masses

    def set_mass(self, proposition, mass):
        """
        Sets mass for a proposition or a singleton
        """
        proposition = [x.lower() for x in proposition]
        index = self._get_index_from_dsvector(proposition)
        self.set_dsvector(index, mass)

    def set_mass_theta(self, mass):
        """
        Sets mass for the full set of all singletons
        """
        index = self._get_index_from_dsvector(self.singletons)
        self.set_dsvector(index, mass)

    def belief(self, proposition):
        """
        Returns belief of a proposition
        """
        proposition = [x.lower() for x in proposition]
        non_zero_subsets = self._get_subsets_from_dsvector(proposition)
        belief = 0
        for subset in non_zero_subsets:
            mass = self.get_mass(subset)
            belief += mass
        return belief / self._get_normalizing_constant()

    def plausibility(self, proposition):
        """
        Returns plausibility of a proposition
        """
        proposition = [x.lower() for x in proposition]
        non_zero_overlaps = self._get_intersections_from_dsvector(proposition)
        plausibility = 0
        for overlapping in non_zero_overlaps:
            mass = self.get_mass(overlapping)
            plausibility += mass
        return plausibility / self._get_normalizing_constant()

    def conditional_mass(self, proposition_b, proposition_a):
        """
        Returns conditional mass (b given a)

        m(proposition_b | proposition_a)
        """
        proposition = [x.lower() for x in proposition]
        subsets_a = self._get_subsets_from_dsvector(proposition_a)
        mass_b = self.get_mass(proposition_b)
        pl_a_minus_b = 0.0
        if not mass_b == 0:
            temp = copy.copy(proposition_a)
            if temp:
                pl_a_minus_b = self.plausibility(temp)

        mass_b_given_a = mass_b / (mass_b + pl_a_minus_b)
        return mass_b_given_a

    def update(self, new_frame, alpha):
        """
        CUE Algorithm
        Mass-based conditional update
        """

        for prop_idx_b, mass_b in new_frame.get_normalized_dsvector().items():

            # conditionall in new_frame on prop_idx_a
            total = 0
            for prop_idx_a, mass_a in new_frame.get_normalized_dsvector().items():

                prop_a = new_frame._get_prop_from_dsvector(prop_idx_a)
                prop_b = new_frame._get_prop_from_dsvector(prop_idx_b)

                mass_b_given_a = 0.0
                if set(prop_a).issubset(set(prop_b)):
                    mass_b_given_a = new_frame.conditional_mass(prop_b, prop_a)

                beta = mass_a
                mult = mass_b_given_a * beta
                total += mult

            prop_b = self._get_prop_from_dsvector(prop_idx_b)
            current_mass = self.get_mass(prop_b)
            term1 = 0.0
            if not current_mass == 0:
                term1 = alpha * current_mass
            term2 = (1 - alpha) * total
            self.set_mass(prop_b, term1 + term2)

    # ------------- SPECIALIZED DS HELPERS -------------------------

    def _initialize_power(self):
        """
        Initializes lookup table called "power".
        Each index i in this lookup table contains the value 2^i.
        The lookup table is of size = number of singletons,
        and represents the position of the singleton masses in massesDSVector
        """
        power = []
        for i in range(self._size_singletons):
            j = 2**i
            power.append(j)
        return power

    def _get_index_from_dsvector(self, proposition):
        """
        Returns a DSVector key of a proposition.
        Useful for various operations including setMass, etc.
        Algorithm obtained from Polpitiya paper 2017.
        """
        proposition = [x.lower() for x in proposition]
        key = 0
        for singleton in proposition:
            try:
                idx = self.singletons.index(singleton)
                key += self.power[idx]
            except:
                raise ValueError(
                    "One of the singletons in the proposition is not found in the frame"
                )
        return key

    def _get_prop_from_dsvector(self, index):
        """
        Returns a proposition given a DSVector index
        """
        indexes = self._find_powers_of_2(index)
        singletons = [self.singletons[i] for i in indexes]
        return singletons

    def _get_subsets_from_dsvector(self, proposition):
        """
        Returns all the subsets of a proposition that have
        non-zero masses

        Note: This does NOT return all subsets, just those
        with non-zero mass in the dsvector
        """
        subsets = []
        for key, value in self.get_dsvector().items():
            key_proposition = self._get_prop_from_dsvector(key)
            if set(key_proposition).issubset(set(proposition)):
                subsets.append(key_proposition)
        return subsets

    def _get_intersections_from_dsvector(self, proposition):
        """
        Returns all non-zero intersections
        """
        overlapping = []
        for key, value in self.get_dsvector().items():
            key_proposition = self._get_prop_from_dsvector(key)
            if set(key_proposition).intersection(set(proposition)):
                overlapping.append(key_proposition)
        return overlapping

    def _get_normalizing_constant(self):
        """
        Returns the sum of masses in the DS-Vector.
        The Masses DSVector is unnormalized and so will
        need to be divided by this normalizing constant.
        """
        return sum(self.get_dsvector().values())

    # ------------- GENERIC HELPERS -------------------------

    def _find_powers_of_2(self, number):
        """
        Returns a list of numbers which when raised to the power of 2
        and added finally, gives the integer number
        """
        indexes = []
        bits = []
        while number > 0:
            bits.append(int(number % 2))
            number = int(number / 2)

        for i in range(0, len(bits)):
            if bits[i] == 1:
                indexes.append(i)
        return indexes

    def _powerset(self, iterable):
        """
        HELPER Function
        Returns power set of an iterable.

        """
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"

        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

# Updated DSFrame

In [315]:
# UPDATED DS
#  v0.2

# singleton: string
# proposition: list of strings

"""
DS-Theoretic Terminology:

theta: Frame of Discernment (FoD) (list of singletons)
mass: a function mapping a subset of the FoD (i.e. a proposition) with number in R
    (a.k.a: basic probability assignment, mass function)
core: F, those propositions that have mass > 0 
focal_elements: Items in F. 
belief: total support for a proposition without any ambiguity (sum of masses of subsets)
plausibility: Extent to which a proposition is plausible (sum of masses of overlapping sets)
"""

from collections import defaultdict
import copy
from itertools import chain, combinations


class BOE:
    """
    A class to represent a DS-Theoretic Body of Evidence
    """

    def __init__(self, singletons):
        """
        Constructor
        """
        # List of lowercased singletons of size n. The index of these singletons is important.
        self._frame = [x.lower() for x in singletons]

        # dict containing masses
        # this is also the masses DSVector
        # initialized to zero
        # UNNORMALIZED
        self._dsvector = defaultdict(self._default_mass)

        # Lookup table containing keys of singletons in the masses_dsvector
        self._power = self._initialize_power()

    def _default_mass(self):
        return 0

    # -------------------------------------
    # DS-Theoretic Properties

    @property
    def frame(self):
        """
        Get singletons or FoD or frame
        List of lowercased singletons of size n.
        """
        return self._frame

    @property
    def power(self):
        """
        Get power:
        Lookup table containing keys of singletons in the masses_dsvector
        """
        return self._power

    @property
    def dsvector(self):
        """
        Get the DSVector
        Dictionary of all subsets of the frame with masses

        Actually, only contains non-zero masses

        It is UNNORMALIZED, which means masses DO NOT add up to 1
        """
        return self._dsvector

    @dsvector.setter
    def dsvector(self, value):
        """
        Set the DSVector
        """
        index, mass = value
        self._dsvector.update({index: mass})

    @property
    def normalizing_constant(self):
        """
        Returns the sum of masses in the DS-Vector.
        The Masses DSVector is unnormalized and so will
        need to be divided by this normalizing constant.
        """
        return sum(self.dsvector.values())

    # ----------------------------------
    # Key DS-Theoretic Operations

    def set_mass(self, proposition, mass):
        """
        Sets mass for a proposition or a singletona

        Just updates the mass in the DSVector.
        Does NOT normalize anything
        """
        proposition = [x.lower() for x in proposition]
        index = self._get_index_from_dsvector(proposition)
        self.dsvector = (index, mass)

    def set_mass_theta(self, mass):
        """
        Sets mass for the frame
        """
        index = self._get_index_from_dsvector(self.frame)
        self.dsvector = (index, mass)

    def get_mass(self, proposition):
        """
        Get UNNORMALIZED mass for a proposition or a singleton
        """
        proposition = [x.lower() for x in proposition]
        index = self._get_index_from_dsvector(proposition)
        return self.dsvector[index]

    def get_normalized_mass(self, proposition):
        """
        Get normalized mass for a proposition
        """
        proposition = [x.lower() for x in proposition]
        return self.get_mass(proposition) / self.normalizing_constant

    def get_masses(self):
        """
        Returns a dict containing proposition and the masses for each
        """
        masses = {}
        for key, value in self.dsvector.items():
            prop = self._get_prop_from_dsvector(key)
            entry = {str(prop): value}
            masses.update(entry)
        return masses

    def get_normalized_masses(self):
        """
        Returns a dict containing proposition and the NORMALIZED masses for each
        """
        masses = self.get_masses()
        normalized_masses = {}
        for key, value in masses.items():
            nmass = value / self.normalizing_constant
            normalized_masses.update({key: nmass})
        return normalized_masses

    def get_normalized_dsvector(self):
        """
        Returns a dsvector with all the masses normalized
        """
        normalized_dsvector = {}
        for key, value in self.dsvector.items():
            normalized_mass = value / self.normalizing_constant
            normalized_dsvector.update({key: normalized_mass})
        return normalized_dsvector

    def get_core(self):
        """
        Returns the core (As a list of propositions)

        All subsets of the frame that have non-zero mass
        """
        core = []
        indexes = self.get_normalized_dsvector().keys()
        for number in indexes:
            core.append(self._get_prop_from_dsvector(number))
        return core

    def belief(self, proposition):
        """
        Returns belief of a proposition

        Adds masses of subset and then divides by normalizing const.

        """
        proposition = [x.lower() for x in proposition]
        non_zero_subsets = self._get_subsets_from_dsvector(proposition)
        belief = 0
        for subset in non_zero_subsets:
            mass = self.get_mass(subset)
            belief += mass
        return belief / self.normalizing_constant

    def plausibility(self, proposition):
        """
        Returns plausibility of a proposition

        Adds masses of overlapping sets and divides by normalizing const
        """
        proposition = [x.lower() for x in proposition]
        non_zero_overlaps = self._get_intersections_from_dsvector(proposition)
        plausibility = 0
        for overlapping in non_zero_overlaps:
            mass = self.get_mass(overlapping)
            plausibility += mass
        return plausibility / self.normalizing_constant

    def uncertainty(self, proposition):
        """
        Returns the uncertainty interval

        [belief, plausibility]
        """
        proposition = [x.lower() for x in proposition]
        return [self.belief(proposition), self.plausibility(proposition)]

    def get_uncertainties(self):
        """
        Returns uncertainty intervals for all items in core
        """
        uncertainties = {}
        for proposition in self.get_core():
            uncertainty = self.uncertainty(proposition)
            uncertainties.update({str(proposition): uncertainty})
        return uncertainties

    def conditional_mass(self, proposition_b, proposition_a):
        """
        Returns conditional mass (b given a)

        m(proposition_b | proposition_a)

        Works on normalized masses

        """
        proposition_a = [x.lower() for x in proposition_a]
        proposition_b = [x.lower() for x in proposition_b]
        mass_b = self.get_mass(proposition_b)
        pl_a_minus_b = 0.0
        if not mass_b == 0:
            temp = copy.copy(proposition_a)
            if temp:
                pl_a_minus_b = self.plausibility(temp)

        mass_b_given_a = mass_b / (mass_b + pl_a_minus_b)
        return mass_b_given_a

    def update(self, new_frame, alpha):
        """
        CUE Algorithm
        Mass-based conditional update

        alpha: amount of weight on existing knowledge
        """

        for prop_idx_b, mass_b in new_frame.get_normalized_dsvector().items():

            # conditional in new_frame on prop_idx_a
            total = 0
            for prop_idx_a, mass_a in new_frame.get_normalized_dsvector().items():

                prop_a = new_frame._get_prop_from_dsvector(prop_idx_a)
                prop_b = new_frame._get_prop_from_dsvector(prop_idx_b)

                mass_b_given_a = 0.0
                if set(prop_a).issubset(set(prop_b)):
                    mass_b_given_a = new_frame.conditional_mass(prop_b, prop_a)

                beta = mass_a
                mult = mass_b_given_a * beta
                total += mult

            prop_b = self._get_prop_from_dsvector(prop_idx_b)
            current_mass = self.get_normalized_mass(prop_b)
            term1 = 0.0
            if not current_mass == 0:
                term1 = alpha * current_mass
            term2 = (1 - alpha) * total
            self.set_mass(prop_b, term1 + term2)

    def update_stream(self, list_boes, alpha):
        """
        CUE update for a list of boes
        """
        print(f"Unc: {self.get_uncertainties()}")
        for idx, boe in enumerate(list_boes):
            self.update(boe, alpha)
            print(f"Unc at [{idx}]: {self.get_uncertainties()}")

    # ------------- SPECIALIZED DS HELPERS -------------------------

    def _initialize_power(self):
        """
        Initializes lookup table called "power".
        Each index i in this lookup table contains the value 2^i.
        The lookup table is of size = number of singletons,
        and represents the position of the singleton masses in massesDSVector
        """
        power = []
        for i in range(len(self.frame)):
            j = 2**i
            power.append(j)
        return power

    def _get_index_from_dsvector(self, proposition):
        """
        Returns a DSVector key of a proposition.
        Useful for various operations including setMass, etc.
        Algorithm obtained from Polpitiya paper 2017.
        """
        proposition = [x.lower() for x in proposition]
        key = 0
        for singleton in proposition:
            try:
                idx = self.frame.index(singleton)
                key += self.power[idx]
            except:
                raise ValueError(
                    "One of the singletons in the proposition is not found in the frame"
                )
        return key

    def _get_prop_from_dsvector(self, index):
        """
        Returns a proposition given a DSVector index
        """
        indexes = self._find_powers_of_2(index)
        singletons = [self.frame[i] for i in indexes]
        return singletons

    def _get_subsets_from_dsvector(self, proposition):
        """
        Returns all the subsets of a proposition that have
        non-zero masses

        Note: This does NOT return all subsets, just those
        with non-zero mass in the dsvector
        """
        subsets = []
        for key, value in self.dsvector.items():
            key_proposition = self._get_prop_from_dsvector(key)
            if set(key_proposition).issubset(set(proposition)):
                subsets.append(key_proposition)
        return subsets

    def _get_intersections_from_dsvector(self, proposition):
        """
        Returns all non-zero intersections
        """
        overlapping = []
        for key, value in self.dsvector.items():
            key_proposition = self._get_prop_from_dsvector(key)
            if set(key_proposition).intersection(set(proposition)):
                overlapping.append(key_proposition)
        return overlapping

    # -------------------- COMBINATION -------------------------

    def conjunctive_form(self, another_boe, proposition):
        """
        m(self inntersection other_boe)
        """
        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        mass = 0.0
        for index1, mass1 in self.get_normalized_dsvector().items():
            for index2, mass2 in another_boe.get_normalized_dsvector().items():
                prop1 = self._get_prop_from_dsvector(index1)  # FIXME-- underscore
                prop2 = another_boe._get_prop_from_dsvector(index2)
                if list(set(prop1) & set(prop2)) == proposition:
                    mass += mass1 * mass2
        return mass

    def disjunctive_form(self, another_boe, proposition):
        """
        m(self union other_boe)
        """
        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        mass = 0.0
        for index1, mass1 in self.get_normalized_dsvector().items():
            for index2, mass2 in another_boe.get_normalized_dsvector().items():
                prop1 = self._get_prop_from_dsvector(index1)  # FIXME-- underscore
                prop2 = another_boe._get_prop_from_dsvector(index2)
                if list(set(prop1) | set(prop2)) == proposition:
                    mass += mass1 * mass2
        return mass

    def conflict(self, another_boe):
        """
        K (or conflict) for Combination rules
        """
        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        conflict = self.conjunctive_form(another_boe, [])
        if conflict == 1:
            return 1

        return conflict

    def dcr(self, another_boe, proposition):
        """
        Dempster's rule of combination
        """
        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        if proposition == []:
            return 0

        if self.conflict(another_boe) == 1:
            return None

        return self.conjunctive_form(another_boe, proposition) / (
            1 - self.conflict(another_boe)
        )

    def yager(self, another_boe, proposition):
        """
        Yager Combination rule
        """
        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        if proposition == []:
            return 0

        if proposition == another_boe.frame:
            return self.conjunctive_form(another_boe, proposition) + self.conflict(
                another_boe
            )

        return self.conjunctive_form(another_boe, proposition)

    def dubois_prade(self, another_boe, proposition):
        """
        Dubois and Prade

        Dubois and Prade proposed to distribute conflict among propositions that actually
        contribute to the conflict.

        """
        if proposition == []:
            return 0

        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        term1 = self.conjunctive_form(another_boe, proposition)

        term2 = 0.0
        for index1, mass1 in self.get_normalized_dsvector().items():
            for index2, mass2 in another_boe.get_normalized_dsvector().items():
                prop1 = self._get_prop_from_dsvector(index1)  # FIXME-- underscore
                prop2 = another_boe._get_prop_from_dsvector(index2)
                if list(set(prop1) | set(prop2)) == proposition:
                    if list(set(prop1) & set(prop2)) == []:
                        term2 += mass1 * mass2

        return term1 + term2

    def pcr5(self, another_boe, proposition):
        """
        Partial Conflict Redistribution (PCR5)

        Smarandache and Dezert

        distribute the partial conflicts among the focal elements
        involved in the conflict.
        """
        proposition = [x.lower() for x in proposition]

        if proposition == []:
            return 0

        if self.frame != another_boe.frame:
            print("Cannot handle non-identical BOEs")
            return None

        term1 = self.conjunctive_form(another_boe, proposition)

        term2 = 0.0
        powers = self._powerset(self.frame)

        for item in powers:
            if list(set(proposition) & set(item)) == []:
                term2 += self.gamma(another_boe, proposition, item)

        return term1 + term2

    def gamma(self, another_boe, proposition1, proposition2):
        """
        Gamma(B,C) useful for PCR
        """

        term1_numerator = (
            self.get_normalized_mass(proposition1) ** 2
        ) * another_boe.get_normalized_mass(proposition2)

        denominator1 = self.get_normalized_mass(
            proposition1
        ) + another_boe.get_normalized_mass(proposition2)

        term2_numerator = (
            another_boe.get_normalized_mass(proposition1) ** 2
        ) * self.get_normalized_mass(proposition2)

        denominator2 = another_boe.get_normalized_mass(
            proposition1
        ) + self.get_normalized_mass(proposition2)

        ratio1 = 0
        if denominator1 != 0:
            ratio1 = term1_numerator / denominator1

        ratio2 = 0
        if denominator2 != 0:
            ratio2 = term2_numerator / denominator2

        return ratio1 + ratio2

    def pcr5_multisource(self, list_boes):
        """
        Returns a fused BOE by repeatedly calling pcr5()

        Wickramaratne reports that PCR5 is NOT
        - associative
        - idempotent
        - cannot handle non-exhaustive FoDs.
        """
        powers = list(self._powerset(self.frame))

        boe1 = copy.copy(self)
        # Doing PCR in pairs
        for idx, boe2 in enumerate(list_boes):
            new_boe = BOE(self.frame)
            for proposition in powers:
                new_boe.set_mass(proposition, boe1.pcr5(boe2, proposition))
            boe1 = new_boe
        return boe1

    # ------------- GENERIC HELPERS -------------------------

    def _find_powers_of_2(self, number):
        """
        Returns a list of numbers which when raised to the power of 2
        and added finally, gives the integer number
        """
        indexes = []
        bits = []
        while number > 0:
            bits.append(int(number % 2))
            number = int(number / 2)

        for i in range(0, len(bits)):
            if bits[i] == 1:
                indexes.append(i)
        return indexes

    def _powerset(self, iterable):
        """
        HELPER Function
        Returns power set of an iterable.

        """
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"

        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


# Todo
## Implement PCR5 procedure
## Implement example test cases.

In [327]:
# Exemplary COMBINATION Test cases


def test_combination_example1():
    """
    Example 1
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.001

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.6)
    boe1.set_mass(["b"], 0.3)
    boe1.set_mass(["a", "b"], 0.1)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.5)
    boe2.set_mass(["b"], 0.2)
    boe2.set_mass(["a", "b"], 0.3)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0.53) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0.17) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["a", "b"]) - 0.03) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.27) < THRESHOLD
    print("Example 1 passed")


def test_combination_example2():
    """
    Example 2
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.2)
    boe1.set_mass(["b"], 0.8)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.9)
    boe2.set_mass(["b"], 0.1)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0.18) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0.08) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["a", "b"]) - 0) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.74) < THRESHOLD

    # DCR
    assert abs(boe1.dcr(boe2, ["a"]) - 0.69) < THRESHOLD
    assert abs(boe1.dcr(boe2, ["b"]) - 0.3) < THRESHOLD

    print("Example 2 passed")


def test_zadeh():
    """
    Example 3 (Zadeh example)
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b", "c"])
    boe1.set_mass(["a"], 0.9)
    boe1.set_mass(["c"], 0.1)

    boe2 = BOE(["a", "b", "c"])
    boe2.set_mass(["b"], 0.9)
    boe2.set_mass(["c"], 0.1)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["c"]) - 0.01) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.99) < THRESHOLD

    # DCR
    assert abs(boe1.dcr(boe2, ["c"]) - 1) < THRESHOLD

    print("Example 3 passed")


def test_combination_example4():
    """
    Example 4 (with total conflict)
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b", "c", "d"])
    boe1.set_mass(["a"], 0.3)
    boe1.set_mass(["c"], 0.7)

    boe2 = BOE(["a", "b", "c", "d"])
    boe2.set_mass(["b"], 0.4)
    boe2.set_mass(["d"], 0.6)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["c"]) - 0) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["d"]) - 0) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 1) < THRESHOLD

    # DCR
    assert boe1.dcr(boe2, ["c"]) == None

    print("Example 4 passed")


def test_combination_example5():
    """
    Example 5
    Convergence to Idempotence
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.7)
    boe1.set_mass(["b"], 0.3)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.7)
    boe2.set_mass(["b"], 0.3)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0.49) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0.09) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.42) < THRESHOLD

    # DCR
    assert abs(boe1.dcr(boe2, ["a"]) - 0.84) < THRESHOLD
    assert abs(boe1.dcr(boe2, ["b"]) - 0.15) < THRESHOLD

    print("Example 5 passed")


def test_combination_example6():
    """
    Example 6
    Majority opinion
    Smarandache, F., & Dezert, J. (2004). A Simple Proportional Conflict Redistribution Rule.
    ArXiv:Cs/0408010. http://arxiv.org/abs/cs/0408010

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.8)
    boe1.set_mass(["b"], 0.2)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.3)
    boe2.set_mass(["b"], 0.7)

    # CONJUNCTIVE FORM
    assert abs(boe1.conjunctive_form(boe2, ["a"]) - 0.24) < THRESHOLD
    assert abs(boe1.conjunctive_form(boe2, ["b"]) - 0.14) < THRESHOLD

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.62) < THRESHOLD

    print("Example 6 passed")


def test_combination_example7():
    """
    Example 1, section 7.3
    Smarandache, F., & Dezert, J. (2005).
    Information fusion based on new proportional conflict redistribution rules.
    2005 7th International Conference on Information Fusion, 8 pp.
    https://doi.org/10.1109/ICIF.2005.1591955

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.6)
    boe1.set_mass(["a", "b"], 0.4)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["b"], 0.3)
    boe2.set_mass(["a", "b"], 0.7)

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.18) < THRESHOLD

    # PCR5
    assert abs(boe1.pcr5(boe2, ["a"]) - 0.54) < THRESHOLD
    assert abs(boe1.pcr5(boe2, ["b"]) - 0.18) < THRESHOLD
    assert abs(boe1.pcr5(boe2, ["a", "b"]) - 0.28) < THRESHOLD

    print("Example 7 passed")


def test_combination_example8():
    """
    Example 2, section 7.3
    Smarandache, F., & Dezert, J. (2005).
    Information fusion based on new proportional conflict redistribution rules.
    2005 7th International Conference on Information Fusion, 8 pp.
    https://doi.org/10.1109/ICIF.2005.1591955

    """
    THRESHOLD = 0.01

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.6)
    boe1.set_mass(["a", "b"], 0.4)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.2)
    boe2.set_mass(["b"], 0.3)
    boe2.set_mass(["a", "b"], 0.5)

    # CONFLICT
    assert abs(boe1.conflict(boe2) - 0.18) < THRESHOLD

    # PCR5
    assert abs(boe1.pcr5(boe2, ["a"]) - 0.62) < THRESHOLD
    assert abs(boe1.pcr5(boe2, ["b"]) - 0.18) < THRESHOLD
    assert abs(boe1.pcr5(boe2, ["a", "b"]) - 0.2) < THRESHOLD

    print("Example 8 passed")


def test_combination_multisources_elaborate():
    """
    This test case lays out the sequence of combinations for 3 sources

    Example from Section 11.4
    "Fusion based on PCR5-approximate"

    Smarandache, F., & Dezert, J. (2005).
    Proportional Conflict Redistribution Rules for Information Fusion.
    ArXiv:Cs/0408064. http://arxiv.org/abs/cs/0408064

    """
    THRESHOLD = 0.1

    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.6)
    boe1.set_mass(["b"], 0.3)
    boe1.set_mass(["a", "b"], 0.1)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.2)
    boe2.set_mass(["b"], 0.3)
    boe2.set_mass(["a", "b"], 0.5)

    boe3 = BOE(["a", "b"])
    boe3.set_mass(["a"], 0.4)
    boe3.set_mass(["b"], 0.4)
    boe3.set_mass(["a", "b"], 0.2)

    boe12 = BOE(["a", "b"])
    boe12.set_mass(["a"], boe1.pcr5(boe2, ["a"]))
    boe12.set_mass(["b"], boe1.pcr5(boe2, ["b"]))
    boe12.set_mass(["a", "b"], boe1.pcr5(boe2, ["a", "b"]))

    boe123 = BOE(["a", "b"])
    boe123.set_mass(["a"], boe12.pcr5(boe3, ["a"]))
    boe123.set_mass(["b"], boe12.pcr5(boe3, ["b"]))
    boe123.set_mass(["a", "b"], boe12.pcr5(boe3, ["a", "b"]))

    assert abs(boe123.get_mass(["a"]) - 0.5) < THRESHOLD
    assert abs(boe123.get_mass(["b"]) - 0.4) < THRESHOLD
    assert abs(boe123.get_mass(["a", "b"]) - 0.01) < THRESHOLD

    print("Example Multi-sources elaborate passed")


def test_combination_multisources():
    """
    This test unlike the above [test_combination_multisources_elaborate()]
    uses the pcr5_multisources function

    The results should be the same as above
    """
    THRESHOLD = 0.1
    boe1 = BOE(["a", "b"])
    boe1.set_mass(["a"], 0.6)
    boe1.set_mass(["b"], 0.3)
    boe1.set_mass(["a", "b"], 0.1)

    boe2 = BOE(["a", "b"])
    boe2.set_mass(["a"], 0.2)
    boe2.set_mass(["b"], 0.3)
    boe2.set_mass(["a", "b"], 0.5)

    boe3 = BOE(["a", "b"])
    boe3.set_mass(["a"], 0.4)
    boe3.set_mass(["b"], 0.4)
    boe3.set_mass(["a", "b"], 0.2)

    list_boes = [boe2, boe3]

    boe123 = boe1.pcr5_multisource(list_boes)

    assert abs(boe123.get_mass(["a"]) - 0.5) < THRESHOLD
    assert abs(boe123.get_mass(["b"]) - 0.4) < THRESHOLD
    assert abs(boe123.get_mass(["a", "b"]) - 0.01) < THRESHOLD

    print("Example Multi-sources passed")


def test_update_example1():
    """
    Testing the CUE based update

    Starting from ignorance, observe what happens with new data
    """
    alpha = 0.5

    boe = BOE(["a", "b"])
    boe.set_mass_theta(1.0)

    s1 = BOE(["a", "b"])
    s1.set_mass(["a"], 1)

    s2 = BOE(["a", "b"])
    s2.set_mass(["a"], 1)

    s3 = BOE(["a", "b"])
    s3.set_mass(["b"], 1)

    stream = [s1, s2, s3]
    boe.update_stream(stream, alpha)

In [328]:
test_combination_example1()
test_combination_example2()
test_zadeh()
test_combination_example4()
test_combination_example5()
test_combination_example6()
test_combination_example7()
test_combination_example8()
test_combination_multisources_elaborate()
test_combination_multisources()
test_update_example1()

Example 1 passed
Example 2 passed
Example 3 passed
Example 4 passed
Example 5 passed
Example 6 passed
Example 7 passed
Example 8 passed
Example Multi-sources elaborate passed
Example Multi-sources passed
Unc: {"['a', 'b']": [1.0, 1.0]}
Unc at [0]: {"['a', 'b']": [1.0, 1.0], "['a']": [0.2, 1.0]}
Unc at [1]: {"['a', 'b']": [1.0, 1.0], "['a']": [0.25925925925925924, 1.0]}
Unc at [2]: {"['a', 'b']": [1.0, 1.0], "['a']": [0.21874999999999997, 0.84375], "['b']": [0.15625, 0.78125]}


In [ ]:
# Test cases

import string
from itertools import chain, combinations
import numpy as np
import random
import pprint
import json


def gen_singletons(n):
    return list(string.ascii_lowercase)[:n]


def powerset(iterable):
    """
    HELPER Function
    Returns power set of an iterable.

     "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    """

    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


def random_masses(n, param=None):
    """
    Generates n random numbers adding to 1

    See https://stackoverflow.com/a/18662466/3263056

    Depending on the main parameter the Dirichlet distribution will
    either give vectors where all the values are close to 1./N where
    N is the length of the vector, or give vectors where most of the
    values of the vectors will be ~0 , and there will be a single 1, or
    give something in between those possibilities.

    EDIT (5 years after the original answer): Another useful fact about
    the Dirichlet distribution is that you naturally get it, if you generate
    a Gamma-distributed set of random variables and then divide them by their sum.
    """
    if not param:
        param = 1

    return np.random.dirichlet(np.ones(n) / param, size=1).round(2)


def gen_frame(n):

    FRAME_SIZE = n
    FOCAL_SIZE = FRAME_SIZE * 2

    # Generate a test frame inputs
    # generating random masses for some propositions
    singletons = gen_singletons(FRAME_SIZE)
    power = list(powerset(singletons))
    powerlist = [list(p) for p in power]
    focal_elements = random.sample(powerlist, k=FOCAL_SIZE)
    masses = list(random_masses(len(focal_elements))[0])
    theta_mass = masses.pop(0)
    if not singletons in focal_elements:
        focal_elements[0] = singletons

    # Make test frame
    boe = BOE(singletons)

    print("Initial BOE")
    print("========")
    print(f"Frame: {boe.frame}")
    print(f"DSVector: {boe.dsvector}")
    print(f"Normalized DSVector: {boe.get_normalized_dsvector()}")

    print("\n")
    print("Updating Masses")
    for (f, m) in zip(focal_elements, masses):
        boe.set_mass(f, m)
    print(f"Core: {boe.get_core()}")
    print(f"DSVector: {json.dumps(boe.dsvector, indent=2)}")

    print("Proposition Indexes")
    for index in boe.dsvector.keys():
        prop = boe._get_prop_from_dsvector(index)
        print(f"{index} ==> {prop}")

    print("\n")
    return boe

    # Todo
    # - Generate test cases

In [ ]:
boe = gen_frame(3)

In [ ]:
belief = boe.uncertainty(["c"])
print(belief)

In [ ]:
boe.get_normalized_dsvector()

In [1]:
from unsure.boe import BOE